## Create CNN Model and Optimize it using Keras Tuner

In [1]:
!pip install keras-tuner

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import cv2
import os

In [3]:
labels = ['Bacterial Pneumonia', 'Corona Virus Disease', 'Normal', 'Tuberculosis', 'Viral Pneumonia']
img_size = 200

def get_training_data(data_dir):
    data = [] 
    
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [5]:
train_images = get_training_data(r'C:\Users\HP\Desktop\Lung Disease Dataset\train')

In [34]:
test_images = get_training_data(r'C:\Users\HP\Desktop\Lung Disease Dataset\test')

In [36]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [17]:
train_images[0].shape

(2,)

In [18]:
train_images = train_images.reshape(len(train_images), 2, 1)
test_images = test_images.reshape(len(test_images), 2, 1)

In [29]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(2, 1, 1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [30]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [31]:
tuner_search=RandomSearch(build_model,
                          objective = 'val_accuracy', 
                          max_trials=5, directory = 'output')

ValueError: Negative dimension size caused by subtracting 3 from 2 for '{{node conv2d_4/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](conv2d_4_input, conv2d_4/Conv2D/ReadVariableOp)' with input shapes: [?,2,1,1], [3,3,1,32].

In [25]:
tuner_search.search(train_images, labels, epochs = 3, validation_split = 0.1)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
96                |?                 |conv_1_filter
3                 |?                 |conv_1_kernel
64                |?                 |conv_2_filter
5                 |?                 |conv_2_kernel
128               |?                 |dense_1_units
0.01              |?                 |learning_rate



ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {"<class 'str'>"})

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        64032     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                1486944   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 1,552,746
Trainable params: 1,552,746
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Train on 54000 samples, validate on 6000 samples
Epoch 4/10
54000/54000 [==============================] - 8s 148us/sample - loss: 0.1383 - accuracy: 0.9486 - val_loss: 0.2772 - val_accuracy: 0.9073
Epoch 5/10
54000/54000 [==============================] - 8s 142us/sample - loss: 0.0999 - accuracy: 0.9637 - val_loss: 0.2729 - val_accuracy: 0.9107
Epoch 6/10
54000/54000 [==============================] - 8s 144us/sample - loss: 0.0719 - accuracy: 0.9734 - val_loss: 0.3503 - val_accuracy: 0.9120
Epoch 7/10
54000/54000 [==============================] - 7s 138us/sample - loss: 0.0520 - accuracy: 0.9811 - val_loss: 0.3996 - val_accuracy: 0.9090
Epoch 8/10
54000/54000 [==============================] - 8s 142us/sample - loss: 0.0396 - accuracy: 0.9860 - val_loss: 0.4146 - val_accuracy: 0.9103
Epoch 9/10
54000/54000 [==============================] - 8s 143us/sample - loss: 0.0336 - accuracy: 0.9881 - val_loss: 0.4527 - val_accuracy: 0.9142
Epoch 10/10
54000/54000 [==========================